## RBF神经网络

看了源码，但是没有自己搭建，有点长....而且这个网络相对比较好理解


In [23]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MSE
import matplotlib.pyplot as pl

## kaggle竞赛框架 ----里面的卷积神经网络还在设计中（还会改进）

In [31]:
import tensorflow as tf
import torch
from torch.utils.data import Dataset
import torchvision
from torch import nn
from torch.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear


# 定义解析函数
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.int64),
        'image_name': tf.io.FixedLenFeature([], tf.string),
    }
    # 解析Example对象
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)

    # 检查特征名称
    if 'image_name' not in parsed_features:
        raise ValueError("The feature 'image_name' is missing from the TFRecord file.")

    # 解码JPEG图像等
    image = tf.image.decode_jpeg(parsed_features['image'], channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    target = parsed_features['target']
    image_name = parsed_features['image_name']

    ##返回值
    return image, target, image_name

# 解析文件中的每个Example
Image, Target, Image_name = [], [], []

# 文件路径
file_path = './ld_train00-1338.tfrec'

# 创建TFRecordDataset对象
dataset = tf.data.TFRecordDataset(file_path)

# 解析
parsed_dataset = dataset.map(parse_example)

# 遍历每个Example并使用数据
for image, target, image_name in parsed_dataset:
    # 在这里使用解析出来的特征
    ##print(f"Image shape: {image.shape}, Target: {target.numpy()}, Image Name: {image_name.numpy()}")
    Image.append(image)
    Target.append(target)
    Image_name.append(image_name)

for n in range(len(Image)):
    Image[n] = Image[n].numpy()
    Image[n] = torch.tensor(Image[n])
    Image[n] = Image[n].reshape([3,512,512])
    Target[n] = Target[n].numpy()
    Target[n] = torch.tensor(Target[n])

OSError: [WinError 182] 操作系统无法运行 %1。 Error loading "D:\Anaconda3\envs\pytest\lib\site-packages\torch\lib\nvfuser_codegen.dll" or one of its dependencies.

In [29]:
# 构造数据集
class CreateDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y

dataset = CreateDataset(Image, Target)
batch_size = 64  ##构建批次
dataset = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

NameError: name 'Dataset' is not defined

In [11]:
file_path_test = './ld_train15-1327.tfrec'
dataset_test = tf.data.TFRecordDataset(file_path_test)
parsed_dataset = dataset_test.map(parse_example)
Image1, Target1, Image_name1 = [], [], []
for image, target, image_name in parsed_dataset:
        # 在这里使用解析出来的特征
        ##print(f"Image shape: {image.shape}, Target: {target.numpy()}, Image Name: {image_name.numpy()}")
        Image1.append(image)
        Target1.append(target)
        Image_name1.append(image_name)
for n in range(len(Image1)):
    Image1[n] = Image[n].numpy()
    Image1[n] = torch.tensor(Image1[n])
    Image1[n] = Image[n].reshape([3,512,512])
    Target1[n] = Target[n].numpy()
    Target1[n] = torch.tensor(Target[n])
dataset_test = CreateDataset(Image1, Target1)
dataset_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

NameError: name 'tf' is not defined

In [13]:
# 模型
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui,self).__init__()
        self.model1 = Sequential(
            MaxPool2d(2),
            Conv2d(3,64,5,1,padding=2),
            MaxPool2d(4),
            Conv2d(64,64,5,1,padding=2),
            MaxPool2d(4),
            Conv2d(64,64,5,1,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(4096,512))
        self.output_model1 = Linear(512,64)
        self.output_model2 = Linear(64,5)

    def forward(self,x):
        x = self.model1(x)
        x = x.relu()
        x = self.output_model1(x)
        x = x.relu()
        x = self.output_model2(x)
        return x

NameError: name 'nn' is not defined

In [ ]:
## cuba设置
device = torch.device("cuda")

## 模型建立
tudui = Tudui()
tudui = tudui.to(device)

## 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

##优化器
optimizer = torch.optim.SGD(tudui.parameters(),lr = 0.01)

##初始值设置
total_train_step = 0
total_test_step = 0
epoch = 10

for i in range(epoch):
    print("---------------第{}轮的开始-------------".format(i+1))

    for data in dataset:
        imgs,targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = tudui(imgs)
        loss = loss_fn(outputs,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step = total_train_step+1
        if total_train_step %100 ==0:
            print("训练次数：{}，Loss:{}".format(total_train_step,loss))

    total_test_loss = 0
    total_accuracy = 0

    with torch.no_grad():
        for data in dataset_test:
            imgs,targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = tudui(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss = total_test_step + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy
    print("整体测试集上的loss：{}".format(total_test_loss))
    print("整体测试集上的正确率：{}".format(total_accuracy/test_data_size))
    total_test_step += 1